In [2]:
%pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/db/7c/9a60add21b96140e22465d9adf09832feade45235cd22f4cb1668a25e443/pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata
  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl (11.3 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py

In [1]:
from openai import OpenAI
import json
import numpy as np
import pandas as pd
import pickle

In [2]:
client = OpenAI(api_key="----")


In [3]:
def load_templates(file_path):
    return pd.read_csv(file_path)

templates_df = load_templates('templates.csv')

In [51]:
# Function to get embeddings from OpenAI
def get_embeddings(text_list, model="text-embedding-ada-002"):
    response = client.embeddings.create(model=model, input=text_list)
    #print(response.data[0])
    return [item.embedding for item in response.data]
    #return res

# Convert templates to text format for embedding
template_texts = templates_df['description'].tolist()
template_metadata = templates_df[['type', 'role']].values.tolist()

# Get embeddings for the templates
template_embeddings = get_embeddings(template_texts)

In [27]:
# Save embeddings and metadata to a file
with open('template_embeddings.pkl', 'wb') as f:
    pickle.dump((template_embeddings, template_metadata), f)

print("Embeddings and metadata saved successfully.")

Embeddings and metadata saved successfully.


In [28]:
# Load embeddings and metadata from file
with open('template_embeddings.pkl', 'rb') as f:
    template_embeddings, template_metadata = pickle.load(f)

# Normalize embeddings
def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=0, keepdims=True)
    return embeddings / norms

# Normalize the template embeddings
template_embeddings = normalize_embeddings(np.array(template_embeddings))


In [52]:
# Function to find the most similar template
def find_best_template(types_roles):
    query_text = " ".join([f"This is a {tr['type']} layout for {tr['role']}." for tr in types_roles])
    query_embedding = get_embeddings([query_text])[0]
    
    # Normalize the query embedding
    query_embedding_np = np.array(query_embedding)
    query_embedding_np = query_embedding_np / np.linalg.norm(query_embedding_np)
    
    # Compute cosine similarity
    dot_product = np.dot(template_embeddings, query_embedding_np)
    best_match_index = np.argmax(dot_product)
    
    best_template_metadata = template_metadata[best_match_index]
    best_template = templates_df[
        (templates_df['type'] == best_template_metadata[0]) &
        (templates_df['role'] == best_template_metadata[1])
    ]['description'].values[0]
    
    return best_template

In [71]:
def enhance_template_with_openai(template, customer_name, types_roles):
    prompt = f"""
    Create a layout for a Customer 360 view for {customer_name} based on the following details:
    {template}

    """

    system_message = f"""
    You are HOLO, a highly advanced AI designed to dynamically generate and enhance UI layouts for a bank.
    Your goal is to create the most efficient and user-friendly Customer 360 views based on provided details.
    You need to reply in a json format with these properties"
    topSection:
      customerInfo:
        components: this is an array of the components used for customerInfo
    mainContentArea:
      layout: one-column or two-column
      sections: this is array of the components displayed in the main contennt area

    For example:
    {{
      "topSection": {{
        "customerInfo": {{
          "components": ["name", "email", "phone"]
        }}
      }},
      "mainContentArea": {{
        "layout": "two-column",
        "sections": ["accounts", "transactions", "loans"]
      }}
    }}
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_message}, {"role": "user", "content": prompt}],
        max_tokens=250,
        n=1,
        stop=None,
        temperature=0.7,
    )

    print("holo response is ", response.choices[0].message.content)
    enhanced_layout = response.choices[0].message.content
    return json.loads(enhanced_layout)



In [77]:
# Test the function
types_roles_test = [
    {"type": "customer", "role": "VIP"},
    {"type": "bank", "role": "bank manager"},
    {"type": "screen", "role": "ipad"}
]

# Call the function and print the result
best_template_result = find_best_template(types_roles_test)
print(json.dumps(best_template_result, indent=4))
enhanced_template_result = enhance_template_with_openai(best_template_result, "Naomi Payton", types_roles_test)
#print(enhanced_template_result, indent=4))



"Bank manager layout. Top section includes: Name, Status, ID, Last Interaction. Main content area (two-column): Account Overview, Team Performance, High Value Transactions, Quick Links."
holo response is  {
  "topSection": {
    "customerInfo": {
      "components": ["name", "status", "id", "lastInteraction"]
    }
  },
  "mainContentArea": {
    "layout": "two-column",
    "sections": ["accountOverview", "teamPerformance", "highValueTransactions", "quickLinks"]
  }
}


In [86]:
# Print similarities for debugging
def print_similarities(types_roles):
    query_text = json.dumps(types_roles)
    query_embedding = get_embeddings([query_text])[0]
    
    query_embedding_np = np.array(query_embedding)
    query_embedding_np = query_embedding_np / np.linalg.norm(query_embedding_np)
    
    dot_product = np.dot(template_embeddings, query_embedding_np)
    
    print("Similarities:", dot_product)

print_similarities(types_roles_test)

Similarities: [ 0.00715392 -0.04976986 -0.00179182 ... -0.01311106  0.0054557
  0.03976775]
